In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import qr as scipy_qr
from scipy.sparse.linalg import LinearOperator

In [8]:
# Problem parameters
d = 100
m = 3*100

# Make matrix
#np.random.seed(0)
Atilde = np.random.normal(size=(d,d))
Atilde = Atilde.T @ Atilde + 1e-3

A = LinearOperator(dtype=None, shape=Atilde.shape, matvec=lambda x: Atilde @ x, rmatvec=lambda x: Atilde @ x)

# Make S and G
S = np.random.choice([-1, 1], size=(d, int(m/3)))
G = np.random.choice([-1, 1], size=(d, int(m/3)))

# Do QR decomp
Q, R = scipy_qr(A @ S, mode="economic")

# Compute approximate trace
term1 = np.trace(Q.T @ ( A @ Q ) )
tmp =  A @ ( G - ( Q @ ( Q.T @ G ) ) )
tmp2 = G.T @ ( tmp - Q @ ( Q.T @ tmp ) )
term2 = (3/m)*np.trace(tmp2)
trace_estimate = term1 + term2

In [63]:
# Make S and G
S = np.random.choice([-1, 1], size=(d, int(m/3)))
G = np.random.choice([-1, 1], size=(d, int(m/3)))

# Do QR decomp
Q, R = scipy_qr(A @ S, mode="economic")

# Compute approximate trace
term1 = np.trace((Q.T @ A @ Q)) 
tmp =  A @ ( G - ( Q @ ( Q.T @ G ) ) )
tmp2 = G.T @ ( tmp - Q @ ( Q.T @ tmp ) )
term2 = (3/m)*np.trace(tmp2)
trace_estimate = term1 + term2

9869.615962730915

In [9]:
def hutch_plus_plus_trace(A, sample_size=30, method="rademacher"):
    """Computes the Hutchinson randomized estimator of tr(A). A must be SPSD.
    
    Here we compute the estimator with sample_size using blocks of samples of size ceil(sample_size/block_size).
    This helps control memory usage vs. vectorization. We don't throw away any samples, so the estimator may be
    computed with a slightly larger sample size than specified, unless exact_sample_size=True.

    sample_size must be a multiple of 3.
    """

    # Get shape
    n = A.shape[0]

    valid_methods = ["standard_gaussian", "rademacher"]
    assert method in valid_methods, f"method must be one of {valid_methods}"

    assert sample_size % 3 == 0, "sample_size must be a multiple of 3."
    
    if method == "rademacher":
        S = np.random.choice([-1, 1], size=(n, int(sample_size/3)))
        G = np.random.choice([-1, 1], size=(n, int(sample_size/3)))
    elif method == "standard_gaussian":
        S = np.random.normal(size=(n, int(sample_size/3)))
        G = np.random.normal(size=(n, int(sample_size/3)))
    else:
        raise NotImplementedError

    # Do QR decomp
    Q, _ = scipy_qr(A @ S, mode="economic")

    # Compute approximate trace
    term1 = np.trace(Q.T @ ( A @ Q ) )
    tmp =  A @ ( G - ( Q @ ( Q.T @ G ) ) )
    tmp2 = G.T @ ( tmp - Q @ ( Q.T @ tmp ) )
    term2 = (3/m)*np.trace(tmp2)
    trace_estimate = term1 + term2
    
    return trace_estimate

In [ ]:

def hutchinson_epsilon_delta_trace(A, epsilon=0.05, delta=0.05, method="rademacher", block_size=20):
    """Computes an (epsilon, delta)-estimator of trace(A). A must be SPSD. This uses lower-bounds from the literature to pick a sample size 
    for the Hutchinson estimator \hat{tr}(A) such that | \hat{tr}(A) - tr(A) | < epsilon*tr(A) with probability greater than 1 - delta."""
    
    valid_methods = ["standard_gaussian", "rademacher"]
    assert method in valid_methods, f"method must be one of {valid_methods}"

    c = (1.0/(epsilon**2))*np.log(2/delta)

    if method == "standard_gaussian":
        sample_size = int(np.ceil(8*c))
    elif method == "rademacher":
        sample_size = int(np.ceil(6*c))
    else:
        raise NotImplementedError

    return hutchinson_trace(A, sample_size=sample_size, method=method, block_size=block_size)